<a href="https://colab.research.google.com/github/trishnak99/ai_colab_2/blob/master/hw2_problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR10 data set

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## Loading the CIFAR10 data set

In [4]:
from keras.datasets import cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
# one hot encode target values
train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)
train_images, train_labels, test_images, test_labels

(array([[[[ 59,  62,  63],
          [ 43,  46,  45],
          [ 50,  48,  43],
          ...,
          [158, 132, 108],
          [152, 125, 102],
          [148, 124, 103]],
 
         [[ 16,  20,  20],
          [  0,   0,   0],
          [ 18,   8,   0],
          ...,
          [123,  88,  55],
          [119,  83,  50],
          [122,  87,  57]],
 
         [[ 25,  24,  21],
          [ 16,   7,   0],
          [ 49,  27,   8],
          ...,
          [118,  84,  50],
          [120,  84,  50],
          [109,  73,  42]],
 
         ...,
 
         [[208, 170,  96],
          [201, 153,  34],
          [198, 161,  26],
          ...,
          [160, 133,  70],
          [ 56,  31,   7],
          [ 53,  34,  20]],
 
         [[180, 139,  96],
          [173, 123,  42],
          [186, 144,  30],
          ...,
          [184, 148,  94],
          [ 97,  62,  34],
          [ 83,  53,  34]],
 
         [[177, 144, 116],
          [168, 129,  94],
          [179, 142,  87],
   

In [0]:
train_images_norm = train_images.astype('float32')
test_images_norm = test_images.astype('float32')
train_images_norm = train_images_norm / 255.0
test_images_norm = test_images_norm / 255.0

##Create graphs

In [0]:
def graphAcc(history):
  history_dict = history.history
  acc_values = history_dict['acc']
  test_acc_values = history_dict['val_acc']
  epochs_range = range(1, epochs+1)

  plt.plot(epochs_range, acc_values, 'bo', label='Training accuracy')
  plt.plot(epochs_range, test_acc_values, 'ro', label='Test accuracy')
  plt.title('Training and test accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

def graphLoss(history):
  history_dict = history.history
  loss_values = history_dict['loss']
  test_loss_values = history_dict['val_loss']
  epochs_range = range(1, epochs + 1)
  
  plt.plot(epochs_range, loss_values, 'bo', label='Training loss')
  plt.plot(epochs_range, test_loss_values, 'ro', label='Test loss')
  plt.title('Training and test loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

##Build underfit

In [0]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(512, activation='relu', kernel_initializer='he_uniform'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc'])

dg = ImageDataGenerator(horizontal_flip=True)
it_train = dg.flow(train_images_norm, train_labels, batch_size=32)
steps = int(train_images_norm.shape[0] / 64)
history = model.fit_generator(it_train, steps_per_epoch=steps, epochs=50, validation_data=(test_images_norm, test_labels), verbose=1)
loss, acc = model.evaluate(test_images_norm, test_labels, verbose=1)

graphAcc(history)
graphLoss(history)

Epoch 1/50
781/781 [==============================] - 65s 84ms/step - loss: 2.0077 - acc: 0.2708 - val_loss: 1.7018 - val_acc: 0.3924
Epoch 2/50
781/781 [==============================] - 65s 83ms/step - loss: 1.6939 - acc: 0.3822 - val_loss: 1.5245 - val_acc: 0.4533
Epoch 3/50
781/781 [==============================] - 65s 83ms/step - loss: 1.5536 - acc: 0.4404 - val_loss: 1.3883 - val_acc: 0.5028
Epoch 4/50
781/781 [==============================] - 65s 83ms/step - loss: 1.4647 - acc: 0.4718 - val_loss: 1.3487 - val_acc: 0.5155
Epoch 5/50
781/781 [==============================] - 65s 84ms/step - loss: 1.3985 - acc: 0.4942 - val_loss: 1.2822 - val_acc: 0.5504
Epoch 6/50
781/781 [==============================] - 64s 82ms/step - loss: 1.3405 - acc: 0.5221 - val_loss: 1.2252 - val_acc: 0.5622
Epoch 7/50
781/781 [==============================] - 64s 82ms/step - loss: 1.2937 - acc: 0.5376 - val_loss: 1.2006 - val_acc: 0.5642
Epoch 8/50
781/781 [==============================] - 63s 81ms

##Build overfit

In [0]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu', kernel_initializer='he_uniform'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc'])

dg = ImageDataGenerator(horizontal_flip=True)
it_train = dg.flow(train_images_norm, train_labels, batch_size=32)
steps = int(train_images_norm.shape[0] / 64)
history = model.fit_generator(it_train, steps_per_epoch=steps, epochs=50, validation_data=(test_images_norm, test_labels), verbose=1)
loss, acc = model.evaluate(test_images_norm, test_labels, verbose=1)

graphAcc(history)
graphLoss(history)

##Build good fit

In [0]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc'])

dg = ImageDataGenerator(horizontal_flip=True)
it_train = dg.flow(train_images_norm, train_labels, batch_size=32)
steps = int(train_images_norm.shape[0] / 64)
history = model.fit_generator(it_train, steps_per_epoch=steps, epochs=60, validation_data=(test_images_norm, test_labels), verbose=1)
loss, acc = model.evaluate(test_images_norm, test_labels, verbose=1)

graphAcc(history)
graphLoss(history)